In [ ]:
import sys
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
import dill as pickle

training_data = pd.read_csv('data/model_training.csv', sep=',', header=None)

X = training_data.values[:, 1:5]
Y = training_data.values[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=100)

model = DecisionTreeClassifier(criterion="gini", random_state=100, max_depth=3, min_samples_leaf=5)
model.fit(X_train, y_train)


## Save/Pickle Underlying Model as `model.pkl`

In [ ]:
import dill as pickle

model_pkl_path = 'model.pkl'

with open(model_pkl_path, 'wb') as fh:
    pickle.dump(model, fh)

In [ ]:
%%bash

ls -l model.pkl

## Initialize Model by Unpickling Underlying Model

In [ ]:
def initialize_model(model_pkl_path):
    import dill as pickle

    with open(model_pkl_path, 'rb') as fh:
        model = pickle.load(fh)

    return model            

In [ ]:
model = initialize_model(model_pkl_path)
print(model)

## Transform `json` -> `nparray`

In [ ]:
def transform_request(request):
    import ujson
    import numpy as np    
    request_str = request.decode('utf-8')
    request_str = request_str.strip().replace('\n', ',')
    # surround the json with '[' ']' to prepare for conversion
    request_str = '[%s]' % request_str
    request_json = ujson.loads(request_str)

    request_transformed = ([json_line['feature0'], \
                            json_line['feature1'], \
                            json_line['feature2'], \
                            json_line['feature3']] for json_line in request_json)
    return np.array(request_transformed)

## Transform `nparray` -> `json`

In [ ]:
def transform_response(response):
    return ujson.dumps(response.tolist())

In [ ]:
def predict(model, 
            request):

    return model.predict(request)

In [ ]:
class PioModel(object):
    def __init__(self,
                 transform_request,
                 transform_response,
                 model,
                 predict):
        self.transform_request = transform_request
        self.transform_response = transform_response
        self.model = model
        self.predict = predict

In [ ]:
pio_model = PioModel(transform_request,
                     transform_response,
                     model,
                     predict)

## Save/Pickle `PioModel` as `pio_model.pkl`

In [ ]:
import dill as pickle

pio_model_pkl_path = 'pio_model.pkl'

with open(pio_model_pkl_path, 'wb') as fh:
    pickle.dump(pio_model, fh)

In [ ]:
%%bash 

ls -al pio_model.pkl